# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import pandas as pd
from sqlalchemy import create_engine
import re
import numpy as np
import pickle
from collections import defaultdict
from collections import Counter
import unicodedata
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelBinarizer
from sklearn.decomposition import PCA, TruncatedSVD

from gensim.models.doc2vec import TaggedDocument, Doc2Vec

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('disaster_response', engine)  
X = df.iloc[:,0:4]
Y = df.iloc[:,5:40]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t.lower() not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [4]:
Y = Y.drop('child_alone', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [6]:
pipeline.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test['message'])

In [7]:
category_names = list(Y.columns)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = category_names

In [22]:
for i in category_names:
    print(classification_report(y_test[i], y_pred[i]))

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5423
          1       0.80      0.44      0.57      1157

avg / total       0.88      0.88      0.87      6580

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6553
          1       1.00      0.04      0.07        27

avg / total       1.00      1.00      0.99      6580

             precision    recall  f1-score   support

          0       0.74      0.86      0.79      3841
          1       0.74      0.57      0.64      2739

avg / total       0.74      0.74      0.73      6580

             precision    recall  f1-score   support

          0       0.92      0.99      0.96      6037
          1       0.60      0.09      0.16       543

avg / total       0.90      0.92      0.89      6580

             precision    recall  f1-score   support

          0       0.95      1.00      0.97      6229
          1       0.77      0.07 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4]
}

model = GridSearchCV(pipeline, param_grid=parameters, verbose = 5, n_jobs = -1, cv=2, scoring = 'roc_auc')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
model.fit(X_train['message'], y_train)
print("Best: %f using %s" % (model.best_score_, model.best_params_))

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, score=0.7934432096983246, total= 3.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.6min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, score=0.7955468482257044, total= 3.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  9.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, score=0.8034745827085664, total= 5.5min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, score=0.8045407579060148, total= 5.5min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 22.4min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200, score=0.8077783518234741, total= 9.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200, score=0.8125869734825578, total= 9.3min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, score=0.795871942925752, total= 3.3min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=50, score=0.7986815631676302, total= 3.3min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, score=0.8081066333276922, total= 5.1min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=

[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 124.7min finished


Best: 0.814566 using {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 200}


In [15]:
y_pred = model.predict(X_test['message'])

In [16]:
for i in np.arange(0,35):
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

             precision    recall  f1-score   support

          0       0.91      0.98      0.94      5492
          1       0.84      0.51      0.63      1088

avg / total       0.90      0.90      0.89      6580

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6549
          1       1.00      0.03      0.06        31

avg / total       1.00      1.00      0.99      6580

             precision    recall  f1-score   support

          0       0.79      0.85      0.82      3861
          1       0.76      0.68      0.72      2719

avg / total       0.78      0.78      0.78      6580

             precision    recall  f1-score   support

          0       0.92      1.00      0.96      6042
          1       0.80      0.07      0.13       538

avg / total       0.91      0.92      0.89      6580

             precision    recall  f1-score   support

          0       0.95      1.00      0.98      6236
          1       0.81      0.10 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Adding genre as a feature

In this part of the code, I am combining TfIdf vectorization with the binary variable representing the genre of the message. I am not planning to employ this model in the final version of the application as it does not assume to have a genre as an input. This coding part is rather done for the self-education and experimenting with pipelines.

The following code is based on the following articles:

https://towardsdatascience.com/custom-transformers-and-ml-data-pipelines-with-python-20ea2a7adb65

https://stackoverflow.com/questions/50523930/custom-transformers-and-gridsearch-valueerror-in-pipeline


In [4]:
class FeatureSelector( BaseEstimator, TransformerMixin ):
    """class allows me to choose a variable for further transformations"""
    def __init__( self, feature_names ):
        self.feature_names = feature_names 
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
    
    #Method that describes what we need this transformer to do
    def transform( self, X, y = None ):
        return X[ self.feature_names ] 

In [5]:
class LabelBinarizerPipelineFriendly(LabelBinarizer):
    """class allows me to fit the model based on the X input."""
    def fit(self, X, y=None):
        super(LabelBinarizerPipelineFriendly, self).fit(X)
        
    def transform(self, X, y=None):
        return super(LabelBinarizerPipelineFriendly, self).transform(X)

    def fit_transform(self, X, y=None):
        return super(LabelBinarizerPipelineFriendly, self).fit(X).transform(X)

In [6]:
# combining the pipeline binarizing the variable 'genre' and the pipeline vectorizing messages

binarizer_pipeline = Pipeline( steps = [ ('first_selector', FeatureSelector('genre')),
                                         ('binarizer', LabelBinarizerPipelineFriendly()) ])
       
tfidf_pipeline = Pipeline( steps = [ ('second_selector', FeatureSelector('message')),
                                     ('vect', CountVectorizer(tokenizer=tokenize)),
                                     ('tfidf', TfidfTransformer()) ])

full_pipeline = FeatureUnion( transformer_list = [ ('binarizer_pipeline', binarizer_pipeline ), 
                                                   ('tfidf_pipeline', tfidf_pipeline ) ] )

In [7]:
# finally, applying Logistic regression to the transformed data
full_pipeline_m = Pipeline( steps = [ ( 'full_pipeline', full_pipeline),
                                      ( 'clf', MultiOutputClassifier(LogisticRegression(random_state=2, 
                                                                                        solver='liblinear'))) ] )

In [110]:
params = {'clf__estimator__C': [0.001,0.01,0.1,1,10,100]}
model = GridSearchCV(full_pipeline_m, params, n_jobs=-1, cv=2, verbose = 5, scoring ='roc_auc')

In [14]:
full_pipeline_m.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('full_pipeline', FeatureUnion(n_jobs=1,
       transformer_list=[('binarizer_pipeline', Pipeline(memory=None,
     steps=[('first_selector', FeatureSelector(feature_names='genre')), ('binarizer', LabelBinarizerPipelineFriendly(neg_label=0, pos_label=1, sparse_output=False))])), ('tfidf_pipel...te=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))])

In [112]:
model.fit(X_train, y_train)
print("Best: %f using %s" % (model.best_score_, model.best_params_))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] clf__estimator__C=0.001 .........................................
[CV]  clf__estimator__C=0.001, score=0.5782246716355237, total= 1.6min
[CV] clf__estimator__C=0.001 .........................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.4min remaining:    0.0s


[CV]  clf__estimator__C=0.001, score=0.5683026728722013, total= 1.6min
[CV] clf__estimator__C=0.01 ..........................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  4.8min remaining:    0.0s


[CV] . clf__estimator__C=0.01, score=0.7160395285578586, total= 1.6min
[CV] clf__estimator__C=0.01 ..........................................


...

[CV] . clf__estimator__C=0.01, score=0.7068898474359514, total= 1.6min
[CV] clf__estimator__C=0.1 ...........................................


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  9.7min remaining:    0.0s


[CV] .. clf__estimator__C=0.1, score=0.7915739258430287, total= 1.6min
[CV] clf__estimator__C=0.1 ...........................................
[CV] .. clf__estimator__C=0.1, score=0.7946223855935297, total= 1.6min
[CV] clf__estimator__C=1 .............................................
[CV] .... clf__estimator__C=1, score=0.8364084738409863, total= 1.7min
[CV] clf__estimator__C=1 .............................................
[CV] .... clf__estimator__C=1, score=0.8404952130397332, total= 1.6min
[CV] clf__estimator__C=10 ............................................
[CV] ... clf__estimator__C=10, score=0.8318678024690688, total= 1.7min
[CV] clf__estimator__C=10 ............................................
[CV] ... clf__estimator__C=10, score=0.8323638963249798, total= 1.7min
[CV] clf__estimator__C=100 ...........................................
[CV] .. clf__estimator__C=100, score=0.8148278899603496, total= 1.7min
[CV] clf__estimator__C=100 ...........................................
[CV] .

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 29.4min finished


Best: 0.838452 using {'clf__estimator__C': 1}


In [114]:
for i in np.arange(0,34):
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5410
          1       0.81      0.57      0.67      1170

avg / total       0.90      0.90      0.89      6580

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6551
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6580

             precision    recall  f1-score   support

          0       0.79      0.86      0.82      3844
          1       0.77      0.69      0.73      2736

avg / total       0.79      0.79      0.78      6580

             precision    recall  f1-score   support

          0       0.93      0.99      0.96      6054
          1       0.71      0.19      0.30       526

avg / total       0.92      0.93      0.91      6580

             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6243
          1       0.72      0.21 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### Doc2vec + PCA + AdaBoost

Here, I am trying to improve the results of the first classification by combining Doc2vec (instead of tf-idf), Principal Component Analysis, and Adaptive Boosting classification algorithm.

The code is based on the following articles:

https://www.oreilly.com/library/view/applied-text-analysis/9781491963036/ch04.html#idm140629387022352

https://stackoverflow.com/questions/50278744/pipeline-and-gridsearch-for-doc2vec

In [95]:
class TextNormalizer(BaseEstimator, TransformerMixin):
    '''Class normalizes text'''
    def __init__(self, language='english'):
        self.stopwords  = set(nltk.corpus.stopwords.words(language))
        self.lemmatizer = WordNetLemmatizer()
    
    def fit(self, X, y=None):
        return self

    def transform(self, documents):
        
        cleaned_documents = []
        
        for text in documents:
            tokens = word_tokenize(text)
            tokens = [t for t in tokens if t.lower() not in self.stopwords and t.lower() not in string.punctuation]
        
            clean_tokens = []
        
            for tok in tokens:
                clean_tok = self.lemmatizer.lemmatize(tok).lower().strip()
                clean_tokens.append(clean_tok)
                cleaned_doc = ' '.join([str(elem) for elem in clean_tokens]) 
                
            cleaned_documents.append(cleaned_doc)
            
        return pd.Series(cleaned_documents)

In [96]:
class Doc2VecModel(BaseEstimator):
    '''Class transforms messages and gives an output as Doc2Vec numerical representation'''
    def __init__(self, dm=1, vector_size=300, window=2, learning_rate=0.02):
        self.learning_rate = learning_rate
        self.d2v_model = None
        self.vector_size = vector_size
        self.window = window
        self.dm = dm

    def fit(self, raw_documents, y=None):
        # Initialize model
        self.d2v_model = Doc2Vec(vector_size=self.vector_size, window=self.window, dm=self.dm, epochs=20, 
                                 alpha=0.025, min_alpha=0.001, learning_rate = self.learning_rate)
        
        # Tag docs    
        tagged_documents = [
            TaggedDocument(words, ['d{}'.format(idx)])
            for idx, words in enumerate(raw_documents)
        ]
        
        # Build vocabulary
        self.d2v_model.build_vocab(tagged_documents)
        
        # Train model
        self.d2v_model.train(tagged_documents, total_examples=len(tagged_documents), epochs=self.d2v_model.epochs)
        
        return self

    def transform(self, raw_documents):
        X = []
        for index, row in raw_documents.iteritems():
            X.append(self.d2v_model.infer_vector(row))
        X = pd.DataFrame(X, index=raw_documents.index)
        return X

    def fit_transform(self, raw_documents, y=None):
        self.fit(raw_documents)
        return self.transform(raw_documents)

In [103]:
pipeline_doc2vec = Pipeline([
    ('normalizer', TextNormalizer()),
    ('doc2vec', Doc2VecModel()),
    ('pca', PCA(n_components=100)),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators = 200, learning_rate = 0.01)))
])

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline_doc2vec.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('normalizer', TextNormalizer(language=None)), ('doc2vec', Doc2VecModel(dm=1, vector_size=300, window=2)), ('pca', PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.01, n_estimators=200, random_state=None),
           n_jobs=1))])

In [105]:
y_pred = pipeline_doc2vec.predict(X_test['message'])

In [106]:
category_names = list(Y.columns)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = category_names

In [107]:
for i in category_names:
    print(classification_report(y_test[i], y_pred[i]))

             precision    recall  f1-score   support

          0       0.83      1.00      0.91      5489
          1       0.00      0.00      0.00      1091

avg / total       0.70      0.83      0.76      6580

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6552
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6580

             precision    recall  f1-score   support

          0       0.59      1.00      0.74      3855
          1       0.68      0.01      0.01      2725

avg / total       0.63      0.59      0.44      6580

             precision    recall  f1-score   support

          0       0.92      1.00      0.96      6081
          1       0.00      0.00      0.00       499

avg / total       0.85      0.92      0.89      6580

             precision    recall  f1-score   support

          0       0.95      1.00      0.97      6240
          1       0.00      0.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


The Doc2Vec model did not give a better result than the first model

#### Adding text statistics as features

The next endeavour to improve the results assumed adding new features to the model based on the characteristics of the messages. 

In [30]:
new_df = pd.concat([X_train['message'], y_train], axis=1)
df = pd.melt(new_df,id_vars=['message'],var_name='category', value_name='values')

In [31]:
df['total_length'] = df['message'].apply(len)
df['capitals'] = df['message'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
df['num_exclamation_marks'] =df['message'].apply(lambda x: x.count('!'))
df['num_question_marks'] = df['message'].apply(lambda x: x.count('?'))
df['num_punctuation'] = df['message'].apply(lambda x: sum(x.count(w) for w in list(string.punctuation)))
df['num_unique_words'] = df['message'].apply(lambda x: len(set(w for w in x.split())))

In [52]:
df[df['values'] == 1].groupby('category').agg(
    {
         'capitals':'mean',
         'total_length':'mean',
         'num_exclamation_marks': 'mean',
         'num_question_marks': 'mean',
         'num_punctuation': 'mean',
         'num_unique_words': 'mean'
    }
)

,capitals,total_length,num_exclamation_marks,num_question_marks,num_punctuation,num_unique_words
category,,,,,,
aid_centers,9.534247,319.264840,0.022831,0.059361,7.515982,39.844749
aid_related,6.648870,172.372266,0.046671,0.070617,4.652169,24.824191
buildings,7.804114,244.441724,0.059745,0.060725,6.352595,32.652302
clothing,6.803922,238.941176,0.091503,0.071895,7.689542,31.813725
cold,6.907317,256.056098,0.085366,0.034146,6.002439,33.634146
death,8.539911,264.465632,0.019956,0.031042,6.588692,35.145233
direct_report,7.047310,124.914999,0.105277,0.118014,4.332987,20.116194
earthquake,7.000535,169.232210,0.066881,0.138042,4.441948,23.582129
electricity,9.319613,283.762712,0.147700,0.062954,8.058111,35.719128


In [5]:
class text_stats(BaseEstimator, TransformerMixin):    
    '''class returns texts statistics as a dataframe'''
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        
        length = X.apply(len)
        capitals = X.apply(lambda comment: sum(1 for c in comment if c.isupper()))
        exclam_marks =X.apply(lambda x: x.count('!'))
        question_marks = X.apply(lambda x: x.count('?'))
        punctuation = X.apply(lambda x: sum(x.count(w) for w in list(string.punctuation)))
        unique_words = X.apply(lambda x: len(set(w for w in x.split())))
        
        df = pd.concat([length, capitals, exclam_marks, question_marks, punctuation, unique_words], axis=1)
        df.columns = ['length', 'capitals', 'exclam_marks', 'question_marks', 'punctuation',
                                      'unique_words']
        return df

Finally, I made two pipelines: the first one with Adaptive Boosting algorithm as a classifier, and the second one with Random Forests.

In [6]:
pipeline_ada_ts = Pipeline([
    ('features', FeatureUnion([
        
        ('nlp_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        
        ('text_stats', text_stats())
    ])),
    
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [7]:
pipeline_ada_ts.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [8]:
parameters = {'clf__estimator__n_estimators': [100, 200, 300],
              'clf__estimator__learning_rate': [0.01, 0.1, 0.2]}

In [9]:
ada_ts_model = GridSearchCV(pipeline_ada_ts, param_grid=parameters, verbose = 5, n_jobs = -1, cv=2, scoring = 'roc_auc')

In [10]:
ada_ts_model.fit(X_train['message'], y_train)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=100, score=0.7360597147628853, total= 3.8min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=100 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.8min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=100, score=0.7264374500299777, total= 3.8min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=200 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  9.7min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=200, score=0.77044188480111, total= 6.0min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=200 


...

[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=200, score=0.77058472340617, total= 5.9min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=300 


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 23.8min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=300, score=0.7872590202054034, total= 8.0min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=300 
[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=300, score=0.784405341299615, total= 8.2min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100, score=0.8109587607356085, total= 3.9min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=100, score=0.7997035700479141, total= 3.9min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=200 
[CV]  clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=200, score=0.8113351809565801, total= 5.9min
[CV] clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=200 
[CV]  clf__estimator__learning_rate=0.1, clf__estimat

[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 127.7min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [100, 200, 300], 'clf__estimator__learning_rate': [0.01, 0.1, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=5)

In [13]:
print("Best: %f using %s" % (ada_ts_model.best_score_, ada_ts_model.best_params_))

Best: 0.807715 using {'clf__estimator__learning_rate': 0.1, 'clf__estimator__n_estimators': 200}


In [15]:
y_pred = ada_ts_model.predict(X_test['message'])

In [16]:
category_names = list(Y.columns)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = category_names

In [17]:
for i in category_names:
    print(classification_report(y_test[i], y_pred[i]))

             precision    recall  f1-score   support

          0       0.89      0.98      0.94      5475
          1       0.82      0.42      0.56      1105

avg / total       0.88      0.89      0.87      6580

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6535
          1       0.00      0.00      0.00        45

avg / total       0.99      0.99      0.99      6580

             precision    recall  f1-score   support

          0       0.72      0.87      0.79      3851
          1       0.75      0.53      0.62      2729

avg / total       0.73      0.73      0.72      6580

             precision    recall  f1-score   support

          0       0.93      1.00      0.96      6055
          1       0.69      0.12      0.21       525

avg / total       0.91      0.93      0.90      6580

             precision    recall  f1-score   support

          0       0.95      1.00      0.98      6217
          1       0.79      0.17 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
pipeline_rf_ts = Pipeline([
    ('features', FeatureUnion([
        
        ('nlp_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        
        ('text_stats', text_stats())
    ])),
    
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [9]:
pipeline_rf_ts.fit(X_train['message'], y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [13]:
parameters = {
    'clf__estimator__n_estimators': [200, 300],
    'clf__estimator__min_samples_split': [2, 3, 4]
}

rf_ts_model = GridSearchCV(pipeline_rf_ts, param_grid=parameters, verbose = 5, n_jobs = -1, cv=2, scoring = 'roc_auc')

In [14]:
rf_ts_model.fit(X_train['message'], y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200, score=0.816842697260615, total= 8.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 10.2min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200, score=0.8173378095610646, total= 8.8min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=300 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 20.5min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=300, score=0.8176832997957235, total=12.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=300 


...

[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=300, score=0.8199705285890831, total=12.5min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=200 


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 49.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=200, score=0.81973904188176, total= 8.1min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=200, score=0.8183907371417456, total= 8.2min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=300 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=300, score=0.8192431266219343, total=11.4min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=300 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=300, score=0.8219819959205732, total=11.5min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200, score=0.8177213166701901, total= 7.7min
[CV] clf__estimator__min_samples_split=4, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_spl

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 138.2min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [200, 300], 'clf__estimator__min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=5)

In [15]:
print("Best: %f using %s" % (rf_ts_model.best_score_, rf_ts_model.best_params_))

Best: 0.821633 using {'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 300}


In [16]:
y_pred = rf_ts_model.predict(X_test['message'])

In [18]:
category_names = list(Y.columns)
y_pred = pd.DataFrame(y_pred)
y_pred.columns = category_names

In [19]:
for i in category_names:
    print(classification_report(y_test[i], y_pred[i]))

             precision    recall  f1-score   support

          0       0.90      0.99      0.94      5448
          1       0.88      0.48      0.62      1132

avg / total       0.90      0.90      0.89      6580

             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6546
          1       0.00      0.00      0.00        34

avg / total       0.99      0.99      0.99      6580

             precision    recall  f1-score   support

          0       0.80      0.85      0.82      3896
          1       0.76      0.70      0.73      2684

avg / total       0.79      0.79      0.79      6580

             precision    recall  f1-score   support

          0       0.92      1.00      0.96      6047
          1       0.65      0.06      0.11       533

avg / total       0.90      0.92      0.89      6580

             precision    recall  f1-score   support

          0       0.95      1.00      0.97      6227
          1       1.00      0.07 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Random Forest classifier gave the best result and was chosen as a final model for the application.

### 9. Export your model as a pickle file

In [21]:
pkl_filename = "rf_ts_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(rf_ts_model, file)